In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4"
from explorer import Explorer

Additionally, Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/home/xingkun/miniconda3/envs/webshop/lib/python3.8/site-packages/jnius_config.py:87: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_filename
/home/xingkun/miniconda3/envs/webshop/lib/python3.8/site-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#

In [2]:
e = Explorer()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Products loaded.

Keys cleaned.

Attributes loaded.

100%|██████████| 1000/1000 [00:00<00:00, 102250.22it/s]


0 skipped

Loaded 13 goals.


/home/xingkun/miniconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 3.0. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/xingkun/miniconda3/envs/webshop/lib/python3.8/site-packages/werkzeug/urls.py:545: DeprecationWarning: 'werkzeug.urls.URL' is deprecated and will be removed in Werkzeug 3.0. Use the 'urllib.parse' library instead.
  return result_type(scheme, netloc, url, query, fragment)
/home/xingkun/miniconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(
/home/xingkun/miniconda3/envs/webshop/lib/python3.8/site-packages/gym/envs/registration.py:619: UserWarning: WARN: Env check failed with the following message: You must specify an observation space (cf gym.spaces) cf https://github.com/openai/gym/blob/master/gym/spaces/
You

In [3]:
for i in range(20):
    e.explore()

/home/xingkun/miniconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 3.0. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/xingkun/miniconda3/envs/webshop/lib/python3.8/site-packages/werkzeug/urls.py:545: DeprecationWarning: 'werkzeug.urls.URL' is deprecated and will be removed in Werkzeug 3.0. Use the 'urllib.parse' library instead.
  return result_type(scheme, netloc, url, query, fragment)
/home/xingkun/miniconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Start exploring at 2025-11-18_19:43:44
Step 0
Current state url: http://127.0.0.1:3000/<session_id>
Action status: {'has_search_bar': True, 'clickables': ['search']}
Retrieved 2 experiences: [{'action': 'search[x-large red women faux fur lined winter warm jacket coat price:lower than 90]', 'action_path': ['search[x-large red women faux fur lined winter warm jacket coat price:lower than 90]'], 'id': '11-15_14:15:24.809830_9_search[x-large red women faux fur lined winter warm jacket coat price:lower than 90]', 'st': {'html': 'WebShop Search', 'url': 'http://127.0.0.1:3000/<session_id>'}, 'st1': {'html': "Back to Search Page 1 (Total results: 50) Next > B09KP78G37 Women Faux Fur Lined Jacket Coat Winter Warm Thick Fleece Outwear Trench Zipper Plus Size Long Sleeve Plush Overcoat $47.41 to $59.07 B07ZXBGDXF Women's Coat, FORUU Winter Faux Fur Fleece Outwear Warm Lapel Biker Motor Aviator Jacket $21.49 to $24.99 B09MSZ4VS8 Goldweather Women Winter Leggings Printed Warm Fleece Lined Yoga Pan

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Action 'search[x-large red women faux fur lined winter warm jacket coat price:lower than 90]' is taken
Step 1
Current state url: http://127.0.0.1:3000/search_results/<session_id>/<query>/1
Action status: {'has_search_bar': False, 'clickables': ['back to search', 'next >', 'b09kp78g37', 'b07zxbgdxf', 'b09msz4vs8', 'b09glvmlms', 'b09c8ynwwb', 'b000j3hzxs', 'b07dkgjr74']}
Retrieved 3 experiences: [{'action': 'click[b09kp78g37]', 'action_path': ['search[x-large red women faux fur lined winter warm jacket coat price:lower than 90]', 'click[b09kp78g37]'], 'id': '11-18_17:57:02.515434_8_search[x-large red women faux fur lined winter warm jacket coat price:lower than 90]-click[b09kp78g37]', 'st': {'html': "Back to Search Page 1 (Total results: 50) Next > B09KP78G37 Women Faux Fur Lined Jacket Coat Winter Warm Thick Fleece Outwear Trench Zipper Plus Size Long Sleeve Plush Overcoat $47.41 to $59.07 B09KP78G37 Women Faux Fur Lined Jacket Coat Winter Warm Thick Fleece Outwear Trench Zipper Plus Si

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

Action 'click[x-large]' is taken
Step 3
Current state url: http://127.0.0.1:3000/item_page/<session_id>/B09KP78G37/<query>/1/{"size": "x-large"}
Action status: {'has_search_bar': False, 'clickables': ['back to search', '< prev', 'description', 'features', 'reviews', 'buy now', 'black', 'navy', 'red', 'yellow', 'small', 'medium', 'large', 'x-large', 'xx-large', '3x-large']}
Retrieved 2 experiences: [{'action': 'click[red]', 'action_path': ['search[x-large red women faux fur lined winter warm jacket coat price:lower than 90]', 'click[b09kp78g37]', 'click[x-large]', 'click[red]'], 'id': '11-18_17:53:25.707825_8_search[x-large red women faux fur lined winter warm jacket coat price:lower than 90]-click[b09kp78g37]-click[x-large]-click[red]', 'st': {'html': 'Back to Search < Prev color black navy red yellow size small medium large x-large xx-large 3x-large Women Faux Fur Lined Jacket Coat Winter Warm Thick Fleece Outwear Trench Zipper Plus Size Long Sleeve Plush Overcoat Price: $47.41 to $59

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Action 'click[x-large]' is taken
Step 4
Current state url: http://127.0.0.1:3000/item_page/<session_id>/B09KP78G37/<query>/1/{"size": "x-large"}
Action status: {'has_search_bar': False, 'clickables': ['back to search', '< prev', 'description', 'features', 'reviews', 'buy now', 'black', 'navy', 'red', 'yellow', 'small', 'medium', 'large', 'x-large', 'xx-large', '3x-large']}
Retrieved 2 experiences: [{'action': 'click[red]', 'action_path': ['search[x-large red women faux fur lined winter warm jacket coat price:lower than 90]', 'click[b09kp78g37]', 'click[x-large]', 'click[red]'], 'id': '11-18_17:53:25.707825_8_search[x-large red women faux fur lined winter warm jacket coat price:lower than 90]-click[b09kp78g37]-click[x-large]-click[red]', 'st': {'html': 'Back to Search < Prev color black navy red yellow size small medium large x-large xx-large 3x-large Women Faux Fur Lined Jacket Coat Winter Warm Thick Fleece Outwear Trench Zipper Plus Size Long Sleeve Plush Overcoat Price: $47.41 to $59

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Action 'click[red]' is taken
Step 5
Current state url: http://127.0.0.1:3000/item_page/<session_id>/B09KP78G37/<query>/1/{"size": "x-large", "color": "red"}
Action status: {'has_search_bar': False, 'clickables': ['back to search', '< prev', 'description', 'features', 'reviews', 'buy now', 'black', 'navy', 'red', 'yellow', 'small', 'medium', 'large', 'x-large', 'xx-large', '3x-large']}
Retrieved 1 experiences: [{'action': 'click[buy now]', 'action_path': ['search[x-large red women faux fur lined winter warm jacket coat price:lower than 90]', 'click[b09kp78g37]', 'click[x-large]', 'click[red]', 'click[buy now]'], 'id': '11-18_18:17:59.482095_8_search[x-large red women faux fur lined winter warm jacket coat price:lower than 90]-click[b09kp78g37]-click[x-large]-click[red]-click[buy now]', 'st': {'html': 'Back to Search < Prev color black navy red yellow size small medium large x-large xx-large 3x-large Women Faux Fur Lined Jacket Coat Winter Warm Thick Fleece Outwear Trench Zipper Plus Siz

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4"
from explorer import Explorer
e = Explorer()

In [ ]:
len(e.exp_backend.exp_store)

In [ ]:
e.remove_redundant_experiences()

In [ ]:
e.remove_redundant_experiences()